# Slots

Veamos el tema de hacer slots para todo el dataset 

In [12]:
%load_ext autoreload
%autoreload 2
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article, Assignment
from django.contrib.auth.models import User

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


¿Cuáles quedaron fuera?


Los que quedaron fuera: (falta uno creo!)

- Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”
- Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”
- Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”
- Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
- Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño
- El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana
- Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix
- Filminas para todos, menos para Maduro
- Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”
- Intentó violar a su novia y ella lo asesinó a escobazos

Espectacular! Se fueron todas las notas malas!!! :-)

In [24]:
import json

with open("../data/training_ids.json") as f:
    training_ids = json.load(f)
    
training_articles = Article.objects.filter(tweet_id__in=training_ids)

training_articles.count()

for i, art in enumerate(training_articles):
    print(f"{i+1:<2}. ({art.id}) {art.tweet_id} -- {art.user} -- {art.title}")

1 . (8694) 1242447731286777856 -- infobae -- El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón
2 . (8473) 1243147648238059520 -- infobae -- Escándalo en España: fallan los tests de coronavirus fabricados en China
3 . (9004) 1247580175455444993 -- clarincom -- Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa
4 . (9122) 1254951309469519875 -- infobae -- Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad”
5 . (8395) 1263868365468135430 -- LANACION -- Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino
6 . (8750) 1273676406614978561 -- LANACION -- "Evolucionen": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero
7 . (8284) 1277424269153239041 -- clarincom -- Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”
8 . (8744) 1296244483655454720 -- c

/home/jmperez/.local/share/virtualenvs/hatespeech-news-labelling-YFKq_jFG/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'api.batch' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/jmperez/.local/share/virtualenvs/hatespeech-news-labelling-YFKq_jFG/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'api.batchassignment' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)


De INTERVIEW: 

- SI: @infobae: El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 
- SI: @infobae: Escándalo en España: fallan los tests de coronavirus fabricados en China 
- SI: @infobae: Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 
- SI: @LANACION: Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 
- NO: @infobae: Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 


Falta algo de movimientos sociales o algo así...

In [ ]:
for 

## Demo ids

In [6]:

with open("../data/demo_ids.json") as f:
    ids = json.load(f)



demo_articles = Article.objects.filter(tweet_id__in=ids)

print("Collected article_ids")

print([art.tweet_id for art in demo_articles])

for art in demo_articles:
    print("\n"*2, "-"*80)
    
    print(art, "\n"*2,"\n")
    print(art.tweet_id)
    continue
    for i, comm in enumerate(art.comment_set.all()):
        print(f"{i+1:<5} - ",comm.text)


    

Collected article_ids
[1242419653030313984, 1253699106385481728, 1254951309469519875, 1275389016511336448, 1310165736011239424]


 --------------------------------------------------------------------------------
@infobae: Mariana Genesio Peña pasa su cuarentena total con guantes, barbijo y desnuda: “Mi cuerpo es el planeta Tierra” 

 

1242419653030313984


 --------------------------------------------------------------------------------
@LANACION: Coronavirus. Motín en Devoto: "Nos negamos a morir en la cárcel" 

 

1253699106385481728


 --------------------------------------------------------------------------------
@infobae: Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 

 

1254951309469519875


 --------------------------------------------------------------------------------
@infobae: China abrió su feria anual de carne de perro en plena pandemia mundial por coronavirus 

 

1275389016511336448


 -----------------------------------

## Interview Articles


In [7]:
art = Article.objects.all()[0]

In [8]:
type(art).__name__

'Article'

In [9]:
import json
with open("../data/interview_ids.json") as f:
    ids = json.load(f)

interview_articles = Article.objects.filter(tweet_id__in=ids)

print("Collected article_ids")

print([art.tweet_id for art in interview_articles])

for art in interview_articles:
    print("\n"*2, "-"*80)
    
    print(art, "\n"*2,"\n")
    continue
    for i, comm in enumerate(art.comment_set.all()):
        print(f"{i+1:<5} - ",comm.text)


    

Collected article_ids
[1242447731286777856, 1243147648238059520, 1254951309469519875, 1296507657189314566, 1302741866681491456]


 --------------------------------------------------------------------------------
@infobae: El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

 



 --------------------------------------------------------------------------------
@infobae: Escándalo en España: fallan los tests de coronavirus fabricados en China 

 



 --------------------------------------------------------------------------------
@infobae: Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 

 



 --------------------------------------------------------------------------------
@infobae: Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

 



 --------------------------------------------------------------------------------
@LANACION: Luis Novaresio contó cómo será su boda con Braul

In [7]:
import random

all_articles = list(Article.objects.all())

sample_articles = random.sample(all_articles, 10)

In [9]:
for art in sample_articles:
    print("\n"*2, "-"*80)
    print(art.tweet_id, art, "\n"*2, art.metadata, "\n")
    for comm in art.comment_set.all()[:5]:
        print(comm.text)
    
    



 --------------------------------------------------------------------------------
1293660570240143360 @clarincom: “Les pibis”, la expresión con lenguaje inclusivo de un joven funcionario nacional que estalló en las redes 

 {'trolo -"no seas" -"no sean"': 2, '"planero"': 1, 'negro -"en negro" -"Rio" -"peste" -"humor negro"': 1} 

@usuario yo pensé que era  lis pibis.
@usuario Gobierno de mierda ,con ideas de mierda para robar  y a la mierda todo !
@usuario Que mamarracho que son
@usuario Cada que incluya con la vocal que se le antoje, menos la o.
@usuario Todo lo que tocan lo destruyen, también van por la lengua española que es tan rica y maravillosa!!!!


 --------------------------------------------------------------------------------
1235579313224900610 @clarincom: Mauricio Macri fue escrachado en un avión: “Ladrón, arruinaste el país” 

 {'feministas': 26, 'aborteras': 1, 'trolo -"no seas" -"no sean"': 2, '"sucios"': 1, '"una bomba"': 1, '"planero"': 2, 'negro -"en negro" -"Rio" 